In [36]:
import sys
import os
import json
import pandas as pd
import numpy as np
import random
import copy
from datetime import datetime
from collections import Counter, defaultdict

import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append("../")

from src.helpers import io
from src.classes.dataset import Dataset
from src.classes.annotation_set import AnnotationSet
from src.classes.annotation_record import AnnotationRecord
from src.helpers.visualisation import barplot_distribution, plot_confusion_matrix, tabulate_annotation_pair_summary, analyze_pair_annotations

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Research Questions:

Characteristics:
* Topic
* Interaction Features
* Function/Purpose
* Media Format
* Answer forms
* Sensitive Uses
* Languages
* Conversation length/stats

1. How does each each conversation characteristic change over time?

Plot types:
* Stacked Area Chart
* ...


## Load Dataset + Annotations

In [37]:
# FILL IN:
PATH_TO_DATASET = "../data/automatic_annotations_v0/wildchat4k-raw.json"
DATASET_ID = "wildchat_1m"
PATH_TO_ANNOTATIONS_DIR = "../data/automatic_annotations_v0/gpto3mini-json-wildchat"

In [38]:
# Load dataset (w/o annotations)
dataset = Dataset.load(PATH_TO_DATASET)

# Load annotations into dataset
for fpath in io.listdir_nohidden(PATH_TO_ANNOTATIONS_DIR):
    annotation_set = AnnotationSet.load_automatic(path=fpath, source="automatic_v0")
    dataset.add_annotations(annotation_set)


prompt-multi_turn_relationship: 0 / 10127 failed due to invalid annotations.
prompt-interaction_features: 0 / 10127 failed due to invalid annotations.
turn-sensitive_use_flags: 0 / 10127 failed due to invalid annotations.
turn-topic: 1 / 10127 failed due to invalid annotations.
response-interaction_features: 0 / 10127 failed due to invalid annotations.
prompt-function_purpose: 6 / 10127 failed due to invalid annotations.
prompt-media_format: 0 / 10127 failed due to invalid annotations.
response-media_format: 0 / 10127 failed due to invalid annotations.
response-answer_form: 0 / 10127 failed due to invalid annotations.


## Research Question 1: Time analysis for XXX?

In [39]:
# Sample code to bucket the message timestamps by month:
annotations = []
for conv in dataset.data:
    conv_id = conv.conversation_id
    for message in conv.conversation:
        if conv.time:
            # Convert timestamp to datetime object
            dt = datetime.fromisoformat(conv.time)
            bucket = dt.strftime('%Y-%m')
            annotation = AnnotationRecord(
                value=bucket,
                target_id=f"{conv_id}-{message.turn}",
                annotator="time_month"
            )
            annotations.append(annotation)

time_bucket_annotation_set = AnnotationSet(
    source="automatic",
    name="time_month",
    level="message",
    dataset_id=dataset.dataset_id,
    annotations=annotations,
)

dataset.add_annotations(time_bucket_annotation_set)

In [41]:
# This *might* work, but if not, we should fix up `get_joint_distribution` to work?
matrix, annotation_pairs = dataset.get_joint_distribution(
    ("turn_topic", "automatic_v0"),
    ("time_month", "automatic"),
    level="message",
)

Found 10120 items with `automatic_v0-turn_topic`, and 20242 with `automatic-time_month`.
Generated 10120 label-level pairs (at level=message) out of 20254 total items.


In [42]:
matrix

,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
Adult & Illicit Content,58,136,108,65,59,27,33,56,73,35,40,44,27
Art & Design,28,27,15,36,55,48,35,52,52,57,58,97,70
Business & Finances,92,82,63,51,33,56,25,60,40,67,67,50,41
Culture,28,11,27,8,9,10,13,16,24,19,29,42,22
Economics,24,31,9,15,12,9,8,10,16,5,13,8,16
Education,64,40,28,32,34,59,31,47,27,41,59,27,41
Employment & Hiring,17,10,4,7,11,8,2,25,8,27,20,4,7
"Entertainment, Hobbies & Leisure",72,25,58,24,38,29,26,22,51,61,58,72,97
Fantasy / Fiction / Fanfiction,77,49,69,100,74,60,43,146,73,84,79,144,96
Fashion & Beauty,10,23,14,4,5,4,1,8,12,5,8,5,4
